# IMPORTS

In [14]:
from collections import *
from functools import *
from itertools import *
import operator
import sys
import re
import math
import string
import bisect
import parse
import matplotlib.pyplot as plt
import numpy as np
import heapq
from matplotlib.path import Path
import ast
import hashlib
import unittest
import copy
from dataclasses import dataclass, field
import sympy
import networkx as nx
from pyvis.network import Network
from typing import *
from shapely.geometry import Polygon
from enum import StrEnum, Enum
from abc import ABC, abstractmethod

In [18]:
A, B = [], []
with open("big.txt", "r") as f:
    for line in f:
        left, right = map(int, line.split())  # Split the line and convert to integers
        A.append(left)
        B.append(right)
    A.sort()
    B.sort()
    p1 = 0
    for x, y in zip(A, B):
        p1 += abs(x - y)
    p2 = 0
    a = Counter(A)
    b = Counter(B)
    for v in A:
        p2 += v * b[v]
    print("part 1:", p1)
    print("part 2:", p2)

part 1: 2344935
part 2: 27647262


In [ ]:
class Day21Test(unittest.TestCase):
    # testing on this input
    """
    ...........
    ......##.#.
    .###..#..#.
    ..#.#...#..
    ....#.#....
    .....S.....
    .##......#.
    .......##..
    .##.#.####.
    .##...#.##.
    ...........
    """
    def sample_inputs(self):
        self.assertEqual(solve("small.txt"), )
    def test_solve(self):
        for i in range(1, 100):
            self.assertEqual(brute(i), solve(i), f"Solutions don't match for i = {i}")
Day21Test().test_solve()

In [ ]:
with open("big.txt", "r") as f:
    grid = [list(line) for line in f.read().splitlines()]
    R, C = len(grid), len(grid[0])
    print(R, C)
    neigborhood = lambda r, c: [(r + 1, c), (r, c + 1), (r - 1, c), (r, c - 1)]
    in_bounds = lambda r, c: 0 <= r < R and 0 <= c < C
    sr = sc = None
    for r, c in product(range(R), range(C)):
        if grid[r][c] == "S":
            grid[r][c] = "."
            sr, sc = r, c
    def create():
        # cache each row
        rcount = [[0] * 2 for _ in range(R)]
        for r, c in product(range(R), range(C)):
            if grid[r][c] == "#": continue
            if c % 2 == 0:
                rcount[r][0] += 1 # even
            else:
                rcount[r][1] += 1 # odd
        return rcount
    rcount = create()
    queue = deque([(sr, sc)])
    steps = 26501365
    steps = 30
    h = steps
    w = steps
    init_parity = (sr % 2) ^ (steps % 2)
    parity = init_parity
    def brute(r, w):
        ans = 0
        for c in range(sc - w, sc + w + 1, 2):
            if grid[r][c] == "#": continue
            ans += 1
        return ans
    def process(r, parity, w):
        res = 0
        if 0 <= sc - w < C and 0 <= sc + w < C:
            res += brute(r, w)
        # left_width = 
        return res
    res = 0
    exp = [0] * R
    for row in range(sr, sr + h + 1):
        r = row % R
        exp[r] = process(r, parity, w)
        res += process(r, parity, w)
        parity ^= 1
        w -= 1
    parity = init_parity
    w = steps
    for row in range(sr, sr - h - 1, -1):
        r = row % R
        exp[r] = process(r, parity, w)
        res += process(r, parity, w)
        parity ^= 1
        w -= 1
    res -= process(sr, init_parity, steps)
    print(res)
    res = 0
    for s in range(steps):
        nqueue = deque()
        seen = set()
        while queue:
            r, c = queue.popleft()
            for nr, nc in neigborhood(r, c):
                if in_bounds(nr, nc) and grid[nr][nc] != "#" and (nr, nc) not in seen:
                    seen.add((nr, nc))
                    nqueue.append((nr, nc))
        queue = nqueue
        res = len(queue)
    print("===========================================================")
    for r, c in queue:
        grid[r][c] = "O"
    for line in grid:
        print("".join(line))
    for r, c in queue:
        grid[r][c] = "."
    print(res)

131 131
855
...................................................................................................................................
.#.................#....#...........#..........#...#.#....#.........................#...........................#......#...........
....#..............##............#.................#...................#.#.....#..##......#...........#...#............#.....##....
....##..##..........#...........#....#......#.......#.........................#.......#.................#.....................#..#.
.......##....###...#.#......###.#............#..#...............................#......#.........#..#.....#........................
.......###......#..#........................##....#.##..........#................................#....##.........#..........#......
......#.....#..#.............#.............#...#................#...............#.....#.#......#..#.#................#..#.#.#.#....
.......#...#...........................#..#.....................

In [ ]:
steps = 26501365
h = steps
w = 2 * steps + 1
print(f"{h:,}, {w:,}")

26,501,365, 53,002,731


In [ ]:
class Module:
    def __init__(self, name, type, outputs):
        self.name = name
        self.type = type
        self.outputs = outputs
        if self.type == "%":
            self.memory = "off"
        else:
            self.memory = {}
    def __repr__(self):
        return f"name = {self.name}, type = {self.type}, outputs = {self.outputs}, memory = {self.memory}"
flip_flop = parse.compile("%{} -> {}")
conjunction = parse.compile("&{} -> {}")
broad = parse.compile("broadcaster -> {}")
with open("small.txt", "r") as f:
    data = f.read().splitlines()
    modules = {}
    broadcast_targets = []
    for line in data:
        if broad.parse(line):
            outputs = broad.parse(line).fixed[0]
            broadcast_targets = outputs.split(", ")
        elif flip_flop.parse(line):
            name, outputs = flip_flop.parse(line).fixed
            module = Module(name, "%", outputs.split(", "))
            modules[name] = module
        else:
            name, outputs = conjunction.parse(line).fixed
            module = Module(name, "&", outputs.split(", "))
            modules[name] = module
    for name, module in modules.items():
        for output in module.outputs:
            if output in modules and modules[output].type == "&":
                modules[output].memory[name] = "low"
    presses = 1
    lcnt = hcnt = 0
    for _ in range(presses):
        lcnt += 1 # pressed button
        queue = deque([(name, "low") for name in broadcast_targets])
        while queue:
            print(queue)
            name, pulse = queue.popleft()
            if pulse == "low": lcnt += 1
            else: hcnt += 1
            if name not in modules: continue
            if modules[name].type == "%": 
                if pulse == "high": continue
                modules[name].memory = "on" if modules[name].memory == "off" else "off"
                target_pulse = "high" if modules[name].memory == "on" else "low"
                for output in modules[name].outputs:
                    if output in modules and modules[output] == "&":
                        modules[output].memory[name] = target_pulse
                    queue.append((output, target_pulse))
            else:
                print(modules[name].memory)
                target_pulse = "low" if all(val == "high" for val in modules[name].memory.values()) else "high"
                for output in modules[name].outputs:
                    queue.append((output, target_pulse))
    print(lcnt, hcnt)
    print("part 1:", lcnt * hcnt)

deque([('a', 'low'), ('b', 'low'), ('c', 'low')])
deque([('b', 'low'), ('c', 'low'), ('b', 'high')])
deque([('c', 'low'), ('b', 'high'), ('c', 'high')])
deque([('b', 'high'), ('c', 'high'), ('inv', 'high')])
deque([('c', 'high'), ('inv', 'high')])
deque([('inv', 'high')])
{'c': 'low'}
deque([('a', 'high')])
4 4
part 1: 16


In [ ]:
for x, y, z, w in zip(debug[0], debug[1], debug[2], debug[3]):
    print(x, y, z, w)
    dxy = x - y
    dxz = x - z
    dxw = x - w
    dyz = y - z
    dyw = y - w
    dzw = z - w
    print("dxy", dxy, "dxz", dxz, "dxw", dxw, "dyz", dyz, "dyw", dyw, "dzw", dzw)
# x happen 86 more clicks after y 
# x happen 54 more clicks after z
# w happens 34 more clicks after x
# z happen 32 more clicks after y
# w happens 120 clicks after y
# w happens 88 clicks after z
# system of equations I believe
"""
x + y + z + w
3847, 3761, 3793, 3881
"""
print(math.lcm(3847, 3761, 3793, 3881))
7693 - 3846

3846 3760 3792 3880
dxy 86 dxz 54 dxw -34 dyz -32 dyw -120 dzw -88
3846 3760 3792 3880
dxy 86 dxz 54 dxw -34 dyz -32 dyw -120 dzw -88
3846 3760 3792 3880
dxy 86 dxz 54 dxw -34 dyz -32 dyw -120 dzw -88
3846 3760 3792 3880
dxy 86 dxz 54 dxw -34 dyz -32 dyw -120 dzw -88
7693 7521 7585 7761
dxy 172 dxz 108 dxw -68 dyz -64 dyw -240 dzw -176
7693 7521 7585 7761
dxy 172 dxz 108 dxw -68 dyz -64 dyw -240 dzw -176
7693 7521 7585 7761
dxy 172 dxz 108 dxw -68 dyz -64 dyw -240 dzw -176
7693 7521 7585 7761
dxy 172 dxz 108 dxw -68 dyz -64 dyw -240 dzw -176
212986464842911


3847

In [ ]:
first = 19674
second = 19224
third = 19392
fourth = 19853
second - first

-450

In [ ]:
"""
34737120 -> too low
55,407,168,240 -> too low
212,986,464,842,911

Flip-flop modules (prefix %) are either on or off; they are initially off. If a flip-flop module receives a high pulse, it is 
ignored and nothing happens. However, if a flip-flop module receives a low pulse, it flips between on and off. If it was off, 
it turns on and sends a high pulse. If it was on, it turns off and sends a low pulse.
off -> on -> off

Conjunction modules (prefix &) remember the type of the most recent pulse received from each of their connected input modules; 
they initially default to remembering a low pulse for each input. When a pulse is received, the conjunction module first updates 
its memory for that input. Then, if it remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends a high pulse.
"""

# SOLVING

In [ ]:
compiler = parse.Parser("{country_code}{phone:11.11},")
content = "0085212345678901, +85212345678902, (852)12345678903,"

results = compiler.findall(content)

for result in results:
    print(result)

In [ ]:

with open("big.txt", "r") as f:
    data = f.read().splitlines()
    gifts = []
    for i, line in enumerate(data):
        j = line.index(":") + 2
        items = line[j:].split(", ")
        gift = {}
        for item in items:
            name, amt = item.split(": ")
            amt = int(amt)
            gift[name] = amt
        gifts.append(gift)
    target = {"children": 3, "cats": 7, "samoyeds": 2, "pomeranians": 3, "akitas": 0, "vizslas": 0, "goldfish": 5, "trees": 3, "cars": 2, "perfumes": 1}
    for i, partial_gift in enumerate(gifts, start = 1):
        if all(target[key] == partial_gift[key] for key in partial_gift): print(i)


In [ ]:

with open("big.txt", "r") as f:
    data = f.read().splitlines()
    gifts = []
    for i, line in enumerate(data):
        j = line.index(":") + 2
        items = line[j:].split(", ")
        gift = {}
        for item in items:
            name, amt = item.split(": ")
            amt = int(amt)
            gift[name] = amt
        gifts.append(gift)
    target = {"children": 3, "cats": 7, "samoyeds": 2, "pomeranians": 3, "akitas": 0, "vizslas": 0, "goldfish": 5, "trees": 3, "cars": 2, "perfumes": 1}
    for i, partial_gift in enumerate(gifts, start = 1):
        if (
            all(target[key] < partial_gift.get(key, math.inf) for key in ["cats", "trees"]) and
            all(target[key] > partial_gift.get(key, -math.inf) for key in ["pomeranians", "goldfish"]) and
            all(target[key] == partial_gift.get(key, target[key]) for key in ["children", "samoyeds", "akitas", "vizslas", "cars", "perfumes"])
            ): print("part 2:", i)
        if all(target[key] == partial_gift[key] for key in partial_gift): print("part 1:", i)


In [ ]:
def bsf(n):
    pos = 0
    while n % 2 == 0:
        pos += 1
        n >>= 1
    return pos
with open("big.txt", "r") as f:
    data = f.read()
    # n = 272 # part 1
    n = 35_651_584
    data = list(map(int, data))
    while len(data) <= n:
        nxt = [c ^ 1 for c in reversed(data)]
        data.append(0)
        data.extend(nxt)
    data = data[:n]
    m = bsf(n)
    delta = 1
    for _ in range(m):
        for i in range(0, len(data), 2 * delta):
            data[i] = 1 if data[i] == data[i + delta] else 0
        delta <<= 1
    print("".join(map(str, [data[i] for i in range(0, n, delta)])))

In [ ]:
from parse import compile
from collections import defaultdict, deque
import time
from typing import List, Dict

def bfs(src: int, dst: int, adj_list: Dict[int, List[int]]) -> int:
    dist = 0
    queue = deque([src])
    vis = set([src])
    while queue:
        sz = len(queue)
        for _ in range(sz):
            node = queue.popleft()
            if node == dst: return dist
            for nei in adj_list[node]:
                if nei in vis: continue
                queue.append(nei)
                vis.add(nei)
        dist += 1
    return -1

def main():
    with open("big.txt", 'r') as f:
        data = f.read().splitlines()
        pat1 = compile("Valve {} has flow rate={:d}; tunnels lead to valves {}")
        pat2 = compile("Valve {} has flow rate={:d}; tunnel leads to valve {}")
        adj_list = defaultdict(list)
        valve_mask = {}
        flow_rates = {}
        for line in data:
            valve, flow, neighbors = pat1.parse(line) if not isinstance(pat1.parse(line), type(None)) else pat2.parse(line)
            neighbors = map(lambda x: x.strip(','), neighbors.split())
            valve_mask[valve] = 1 << len(valve_mask) # key: valve string name, value: integer representation (2^i) 
            flow_rates[valve] = flow
            adj_list[valve].extend(neighbors)
        valves_nonzero_flow = [k for k, v in flow_rates.items() if v]
        minDist = {}
        for src in ['AA'] + valves_nonzero_flow:
            for dst in valves_nonzero_flow:
                if src == dst: continue
                minDist[(src, dst)] = bfs(src, dst, adj_list)
        total_time = 26
        memo = {}
        res = 0
        states = deque([('AA', 0, 0, 0, False)]) # (valve, opened, pressure, time, elephants_turn)
        while states:
            pos, opened, pressure, time, elephants_turn = states.popleft()
            res = max(res, pressure)
            if not elephants_turn:
                states.append(('AA', opened, pressure, 0, True))
            if time == total_time: continue
            for dst in valves_nonzero_flow4
                mask = valve_mask[dst]
                if (opened & mask) != 0: continue
                dist = minDist[(pos, dst)]
                if total_time-dist-1-time < 0: continue
                next_pressure = pressure + flow_rates[dst]*(total_time-dist-1-time)
                key = (mask, opened|mask, time, elephants_turn) # O(n * 2^n * t * num_players)
                if key in memo and next_pressure <= memo[key]: continue
                memo[key] = next_pressure
                states.append((dst, opened|mask, next_pressure, time+dist+1, elephants_turn))
        return res

if __name__ == '__main__':
    start_time = time.perf_counter()
    print(main())
    end_time = time.perf_counter()
    print(f'Time Elapsed: {end_time - start_time:,} seconds')

In [ ]:
directions = {
    "E": (0, 1),
    "W": (0, -1),
    "N": (-1, 0),
    "S": (1, 0)
}
with open("big.txt", "r") as f:
    data = f.read().splitlines()
    grid = [list(line) for line in data]
    R, C = len(grid), len(grid[0])
    # init_state = (0, 0, "E")
    init_state = (0, 0, "S")
    vis = set([init_state])
    seen = set([(0, 0)])
    q = deque([init_state])
    in_bounds = lambda r, c: 0 <= r < R and 0 <= c < C
    while q:
        r, c, d = q.popleft()
        seen.add((r, c))
        dr, dc = directions[d]
        r += dr
        c += dc
        if not in_bounds(r, c): continue
        ch = grid[r][c]
        print(r, c, d, ch)
        if ch == "|" and d in "EW":
            if (r, c, "N") not in vis:
                vis.add((r, c, "N"))
                q.append((r, c, "N"))
            if (r, c, "S") not in vis:
                vis.add((r, c, "S"))
                q.append((r, c, "S"))
        elif ch == "-" and d in "NS":
            if (r, c, "E") not in vis:
                vis.add((r, c, "E"))
                q.append((r, c, "E"))
            if (r, c, "W") not in vis:
                vis.add((r, c, "W"))
                q.append((r, c, "W"))
        elif ch == "/":
            if d == "E":
                if (r, c, "N") not in vis:
                    vis.add((r, c, "N"))
                    q.append((r, c, "N"))
            elif d == "N":
                if (r, c, "E") not in vis:
                    vis.add((r, c, "E"))
                    q.append((r, c, "E"))
            elif d == "W":
                if (r, c, "S") not in vis:
                    vis.add((r, c, "S"))
                    q.append((r, c, "S"))
            elif d == "S":
                if (r, c, "W") not in vis:
                    vis.add((r, c, "W"))
                    q.append((r, c, "W"))
        elif ch == "\\":
            if d == "E":
                if (r, c, "S") not in vis:
                    vis.add((r, c, "S"))
                    q.append((r, c, "S"))
            elif d == "N":
                if (r, c, "W") not in vis:
                    vis.add((r, c, "W"))
                    q.append((r, c, "W"))
            elif d == "W":
                if (r, c, "N") not in vis:
                    vis.add((r, c, "N"))
                    q.append((r, c, "N"))
            elif d == "S":
                if (r, c, "E") not in vis:
                    vis.add((r, c, "E"))
                    q.append((r, c, "E"))
        else:
            if (r, c, d) not in vis:
                vis.add((r, c, d))
                q.append((r, c, d))
    # print(vis)
    print(len(seen))




In [ ]:
directions = {
    "U": (-1, 0),
    "D": (1, 0),
    "L": (0, -1),
    "R": (0, 1)
}
dirs = "UDLR"
open = "bcdef"
with open("big.txt", "r") as f:
    grid = [list(line) for line in f.read().splitlines()]
    code = "ihgpwlah"
    R, C = len(grid), len(grid[0])
    sr = sc = None
    for r, c in product(range(R), range(C)):
        if grid[r][c] == "S":
            sr, sc = r, c
            break
    seen = set([(sr, sc, code)])
    q = deque([(sr, sc, code)])
    while q:
        r, c, path = q.popleft()
        h = hashlib.md5(path.encode("utf-8")).hexdigest()[:4]
        for i, d in enumerate(dirs):
            if h[i] not in open: continue # locked
            path += dirs[i]
        break

In [ ]:
directions = {
    "E": (0, 1),
    "W": (0, -1),
    "N": (-1, 0),
    "S": (1, 0)
}
with open("big.txt", "r") as f:
    data = f.read().splitlines()
    grid = [list(map(int, line)) for line in data]
    R, C = len(grid), len(grid[0])
    in_bounds = lambda r, c: 0 <= r < R and 0 <= c < C
    memo = defaultdict(lambda: math.inf)
    heap = [(0, 0, 0, 0, "E")]
    while heap:
        cost, r, c, dist, d = heapq.heappop(heap)
        if (r, c) == (R - 1, C - 1): print(cost)
        dr, dc = directions[d]
        nr, nc = r + dr, c + dc
        if not in_bounds(nr, nc): continue
        ncost = cost + grid[nr][nc]
        if ncost < memo[(nr, nc, dist, d)]: 
            memo[(nr, nc, dist, d)] = ncost
            if dist < 2: # go straight
                heapq.heappush(heap, (ncost, nr, nc, dist + 1, d))
            if d in "EW":
                heapq.heappush(heap, (ncost, nr, nc, 0, "N"))
                heapq.heappush(heap, (ncost, nr, nc, 0, "S"))
            else:
                heapq.heappush(heap, (ncost, nr, nc, 0, "E"))
                heapq.heappush(heap, (ncost, nr, nc, 0, "W"))

# for r, c in path:
#     grid[r][c] = "#"

# for line in grid:
#     print("".join(map(str, line)))

In [ ]:
"""
1040
1041
1042
1043

"""

In [ ]:
# f = g + h, cost + heuristic
with open("big.txt", "r") as f:
    data = f.read().splitlines()
    grid = [list(map(int, line)) for line in data]
    R, C = len(grid), len(grid[0])
    in_bounds = lambda r, c: 0 <= r < R and 0 <= c < C
    memo = set()
    heap = [(R + C, 0, 0, 0, 0, 0, 3)]
    def g(cost, r, c):
        return cost + grid[r][c]
    while heap:
        f, cost, r, c, dr, dc, steps = heapq.heappop(heap)
        if (r, c, dr, dc, steps) in memo: continue
        memo.add((r, c, dr, dc, steps))
        if (r, c) == (R - 1, C - 1): print(cost); break
        if steps < 3:
            nr, nc = r + dr, c + dc
            if in_bounds(nr, nc):
                heapq.heappush(heap, (R + C - nr - nc + g(cost, nr, nc), g(cost, nr, nc), nr, nc, dr, dc, steps + 1))
        for ndr, ndc in [(0, 1), (1, 0), (-1, 0), (0, -1)]:
            if (ndr, ndc) == (dr, dc) or (ndr, ndc) == (-dr, -dc): continue
            nr, nc = r + ndr, c + ndc
            if in_bounds(nr, nc):
                heapq.heappush(heap, (R + C - nr - nc + g(cost, nr, nc), g(cost, nr, nc), nr, nc, ndr, ndc, 1))

In [ ]:
with open("big.txt", "r") as f:
    data = f.read().splitlines()
    grid = [list(map(int, line)) for line in data]
    R, C = len(grid), len(grid[0])
    in_bounds = lambda r, c: 0 <= r < R and 0 <= c < C
    memo = set()
    heap = [(0, 0, 0, 0, 0, 10)]
    while heap:
        cost, r, c, dr, dc, steps = heapq.heappop(heap)
        if (r, c, dr, dc, steps) in memo: continue
        memo.add((r, c, dr, dc, steps))
        if steps >= 4 and (r, c) == (R - 1, C - 1): print(cost); break
        if steps < 10: # cannot go straight after 10 steps
            nr, nc = r + dr, c + dc
            if in_bounds(nr, nc):
                heapq.heappush(heap, (cost + grid[nr][nc], nr, nc, dr, dc, steps + 1))
        if steps >= 4: # minimum of 4 steps before turns
            for ndr, ndc in [(0, 1), (1, 0), (-1, 0), (0, -1)]:
                if (ndr, ndc) == (dr, dc) or (ndr, ndc) == (-dr, -dc): continue
                nr, nc = r + ndr, c + ndc
                if in_bounds(nr, nc):
                    heapq.heappush(heap, (cost + grid[nr][nc], nr, nc, ndr, ndc, 1))

In [ ]:
turn_on = compile("turn on {:d},{:d} through {:d},{:d}")
turn_off = compile("turn off {:d},{:d} through {:d},{:d}")
toggle = compile("toggle {:d},{:d} through {:d},{:d}")
with open('input.txt', 'r') as f:
    data = f.read().splitlines()
    lights = [[0] * 1000 for _ in range(1000)]
    for line in data:
        to, toff, tog = turn_on.parse(line), turn_off.parse(line), toggle.parse(line)
        if to is not None:
            r1, c1, r2, c2 = to.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 1
        elif toff is not None:
            r1, c1, r2, c2 = toff.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] = max(0, lights[r][c] - 1)
        else:
            r1, c1, r2, c2 = tog.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 2
    print(sum(map(sum, lights)))